# PageRank

## 读取数据

In [1]:
import numpy as np

# 读数据
def read_data():
    file = open('Data.txt', 'r')
    graph=[] 
    node_set=set()
    for line in file:
        data=line.split()
        edge=(int(data[0]),int(data[1]))  # 以tuple存储两个点/边
        node_set.add(edge[0])
        node_set.add(edge[1])
        graph.append(edge)

    node_num=len(node_set) # 点的个数
    return graph, node_num, node_set

In [21]:
G, node_num, node_set = read_data()
print('点的个数:',node_num)
# 验证点的序号是否为 1-node_num
for i in range(1,node_num+1):
    if i not in node_set:
        print('点的序号不连续')
        break
print('点的序号连续')

点的个数: 8297
点的序号连续


## test

In [22]:
import networkx as nx

testG = nx.Graph()
for edge in G:
    testG.add_edge(edge[0], edge[1])
print(testG.number_of_nodes())


8297


In [24]:
pr = nx.pagerank(testG, alpha=0.85)
# 对 pagerank 值进行排序
sorted_pr = sorted(pr.items(), key=lambda x: x[1], reverse=True)
# 打印前100个节点
for node, pr_value in sorted_pr[:100]:
    print(f"Node: {node}, PageRank: {pr_value}")
# 计算所有节点pagerank值之和
pr_sum = sum(pr.values())
print(f"所有节点pagerank值之和: {pr_sum}")


Node: 8078, PageRank: 0.0002176009432602875
Node: 853, PageRank: 0.00021594446612430528
Node: 3396, PageRank: 0.00021517393230798196
Node: 2879, PageRank: 0.0002120640899849291
Node: 2006, PageRank: 0.0002077522854063683
Node: 7977, PageRank: 0.0002077051471382337
Node: 2061, PageRank: 0.00020629454669300536
Node: 3742, PageRank: 0.0002051858441447194
Node: 4144, PageRank: 0.0002051500417506352
Node: 5663, PageRank: 0.00020459324607534308
Node: 7540, PageRank: 0.00020456606667758846
Node: 2057, PageRank: 0.000204085379955003
Node: 1657, PageRank: 0.00020401155260374356
Node: 3314, PageRank: 0.00020280806849110622
Node: 4358, PageRank: 0.00020279765741826837
Node: 7887, PageRank: 0.0002027610553192458
Node: 4682, PageRank: 0.0002027240565580038
Node: 2013, PageRank: 0.00020167586843281458
Node: 6211, PageRank: 0.0002014662703749108
Node: 2148, PageRank: 0.0002000060255840566
Node: 6083, PageRank: 0.00019990441180653973
Node: 1445, PageRank: 0.00019959395094739025
Node: 7997, PageRank: 0

## 基础版本

In [25]:
# 初始化邻接矩阵
def get_stochastic_matrix(G, node_num):
	matrix = np.zeros((node_num,node_num))
	# 统计邻接矩阵
	for edge in G:
		matrix[edge[1]-1][edge[0]-1] = 1  # 入度0->1
	# 计算
	for j in range(node_num):
		sum_of_col = sum(matrix[:,j])  # 出度之和(d)
		# 如果发现dead-end，将其转为随机跳转
		if sum_of_col == 0:
			matrix[:,j] = 1/node_num
			continue
		for i in range(node_num):  # 1/d
			matrix[i,j] /= sum_of_col
	return matrix

# 迭代
def power_interation(matrix, beta, node_num):
	# 用 1/node_num 初始化rank vector
	scores = np.ones((node_num))/node_num
	new_scores = np.zeros((node_num))
	interation_num = 0  # 迭代次数
	e = node_num # 两次迭代之间的误差
	while e > 1e-6:
		new_scores = beta*np.dot(matrix,scores)+(1-beta)/node_num  # β随机游走
		e = sum(abs(new_scores-scores))
		scores = np.copy(new_scores)
		interation_num += 1
	return scores, interation_num

In [33]:
beta = 0.85  # 按照链接跳转的概率
matrix = get_stochastic_matrix(G, node_num)
scores, interation_num = power_interation(matrix, beta, node_num)
print('PageRank:', scores)
print('迭代次数:', interation_num)

PageRank: [5.39597744e-04 9.59161392e-05 1.12610782e-04 ... 9.50066677e-05
 8.35763251e-05 7.36008357e-05]
迭代次数: 53


In [34]:
print(sum(scores))
# 只取最大的前100个
sorted_indices = np.argsort(scores)[::-1][:100]
sorted_scores = scores[sorted_indices]
print('Top 100:', sorted_scores)
print('Top 100的点:', sorted_indices+1)  # 点的序号从1开始

0.9999999999999979
Top 100: [0.00087185 0.00085453 0.00084961 0.0008359  0.00083059 0.00082064
 0.00081788 0.00081033 0.00080999 0.000806   0.00080509 0.00080316
 0.00080222 0.00079232 0.00078217 0.0007812  0.00077737 0.00077438
 0.00076882 0.00076079 0.00075903 0.00075754 0.00075585 0.00075166
 0.00074796 0.0007474  0.00074577 0.00074494 0.00074458 0.00074116
 0.0007401  0.00073804 0.00073637 0.00073385 0.00073313 0.00073267
 0.00073254 0.00073134 0.00073048 0.00072626 0.00072583 0.00072582
 0.00072123 0.00071736 0.00071543 0.00071399 0.00071396 0.00071301
 0.00071236 0.00071202 0.00071025 0.00070964 0.00070934 0.00070916
 0.00070835 0.00070823 0.00070495 0.00070113 0.00069982 0.00069907
 0.00069859 0.00069789 0.00069758 0.0006968  0.00069651 0.00069614
 0.00069586 0.00069517 0.00069411 0.00069328 0.00069273 0.00069209
 0.00069156 0.00069146 0.00069116 0.00069044 0.00068669 0.00068636
 0.0006859  0.00068532 0.00068364 0.00068344 0.00068323 0.00068301
 0.00068281 0.00068277 0.00068233 

In [35]:
with open('result_basic.txt', 'w') as file:
    # 遍历列表的索引和值
    for i in range(len(sorted_indices)):
        # 写入格式化的字符串到文件
        file.write(str(sorted_indices[i]+1) + ' ' + str(sorted_scores[i]) + '\n')

## 稀疏矩阵优化

In [29]:
# 初始化稀疏矩阵
def get_sparse_matrix(G, node_num):
    sparse_matrix = [[] for _ in range(node_num)]
    for edge in G:
        sparse_matrix[edge[0]-1].append(edge[1]-1)  # 出度0->1
    return sparse_matrix

def power_interation_sparse(sparse_matrix, beta, node_num):
    # 用 1/node_num 初始化分数 
    scores = np.ones((node_num))/node_num  # 1/N
    e = node_num  # 两次迭代之间的误差
    interation_num = 0  # 迭代次数
    while e > 1e-6:
        new_scores = (1-beta)*np.ones((node_num))/node_num
        for i in range(node_num):
            # 如果是dead-end
            if len(sparse_matrix[i]) == 0:  # 没有出度，renormalize？
                new_scores += beta*scores[i]/node_num
                continue
            for j in sparse_matrix[i]:  # i->j
                new_scores[j] += beta*scores[i]/len(sparse_matrix[i])
        e = sum(abs(new_scores-scores))
        scores = np.copy(new_scores)
        interation_num += 1
    return scores, interation_num

def power_interation_sparse_book(sparse_matrix, beta, node_num):
    # 用 1/node_num 初始化分数 
    scores = np.ones((node_num))/node_num  # 1/N
    e = node_num  # 两次迭代之间的误差
    interation_num = 0  # 迭代次数
    while e > 1e-6:
        new_scores = np.zeros((node_num))
        for i in range(node_num):
            for j in sparse_matrix[i]:  # i->j
                new_scores[j] += beta*scores[i]/len(sparse_matrix[i])
        # re-insert the leaked PageRank
        new_scores += (1-sum(new_scores))/node_num
        e = sum(abs(new_scores-scores))
        scores = np.copy(new_scores)
        interation_num += 1
    return scores, interation_num

In [30]:
beta = 0.85
sparse_matrix = get_sparse_matrix(G, node_num)
#scores , interation_num = power_interation_sparse(get_sparse_matrix(G, node_num), beta, node_num)
scores , interation_num = power_interation_sparse_book(get_sparse_matrix(G, node_num), beta, node_num)

print('迭代次数:', interation_num)

迭代次数: 53


In [31]:
print(sum(scores))
# 只取最大的前100个
sorted_indices = np.argsort(scores)[::-1][:100]
sorted_scores = scores[sorted_indices]
print('Top 100:', sorted_scores)
print('Top 100的点:', sorted_indices+1)  # 点的序号从1开始

0.9999999999999969
Top 100: [0.00087185 0.00085453 0.00084961 0.0008359  0.00083059 0.00082064
 0.00081788 0.00081033 0.00080999 0.000806   0.00080509 0.00080316
 0.00080222 0.00079232 0.00078217 0.0007812  0.00077737 0.00077438
 0.00076882 0.00076079 0.00075903 0.00075754 0.00075585 0.00075166
 0.00074796 0.0007474  0.00074577 0.00074494 0.00074458 0.00074116
 0.0007401  0.00073804 0.00073637 0.00073385 0.00073313 0.00073267
 0.00073254 0.00073134 0.00073048 0.00072626 0.00072583 0.00072582
 0.00072123 0.00071736 0.00071543 0.00071399 0.00071396 0.00071301
 0.00071236 0.00071202 0.00071025 0.00070964 0.00070934 0.00070916
 0.00070835 0.00070823 0.00070495 0.00070113 0.00069982 0.00069907
 0.00069859 0.00069789 0.00069758 0.0006968  0.00069651 0.00069614
 0.00069586 0.00069517 0.00069411 0.00069328 0.00069273 0.00069209
 0.00069156 0.00069146 0.00069116 0.00069044 0.00068669 0.00068636
 0.0006859  0.00068532 0.00068364 0.00068344 0.00068323 0.00068301
 0.00068281 0.00068277 0.00068233 

In [32]:
with open('result_sparse.txt', 'w') as file:
    # 遍历列表的索引和值
    for i in range(len(sorted_indices)):
        # 写入格式化的字符串到文件
        file.write(str(sorted_indices[i]+1) + ' ' + str(sorted_scores[i]) + '\n')

# 分块优化

In [ ]:
# 初始化稀疏矩阵
sparse_matrix = [[] for _ in range(node_num)]

for edge in G:
    sparse_matrix[edge[0]-1].append(edge[1]-1)

# 用 1/node_num 初始化分数 
scores=np.ones((node_num))/node_num

e=1 # 两次迭代之间的误差

block_size=2000 # 每次迭代的块大小

block_num=node_num//block_size

remainder=node_num%block_size



new_scores=np.zeros((node_num))

while e>1e-5:
    # 每次处理一块
    e=0
    for i in range(block_num):
        # 初始化该块
        new_scores[i*block_size:(i+1)*block_size]=(1-beta)/node_num
        # 遍历稀疏矩阵
        for j in range(node_num):
            # 遇到dead-end
            if len(sparse_matrix[j])==0:
                new_scores[i*block_size:(i+1)*block_size]+=beta*scores[j]/node_num
                continue
            
            for m in sparse_matrix[j]:
                if m>=i*block_size and m<(i+1)*block_size:
                    new_scores[m]+=beta*scores[j]/len(sparse_matrix[j])
        e+=sum(abs(new_scores[i*block_size:(i+1)*block_size]-scores[i*block_size:(i+1)*block_size]))
    
    
    
    # 处理剩余部分
    new_scores[block_num*block_size:]=(1-beta)/node_num

    for j in range(node_num):
        if len(sparse_matrix[j])==0:
            new_scores[block_num*block_size:]+=beta*scores[j]/node_num
            continue
        for m in sparse_matrix[j]:
            if m>=block_num*block_size:
                new_scores[m]+=beta*scores[j]/len(sparse_matrix[j])
    e+=sum(abs(new_scores[block_num*block_size:]-scores[block_num*block_size:]))
    
    scores=np.copy(new_scores)


print(scores)

In [ ]:
print(sum(scores))
print(node_num)
print(sum(scores))
sorted_indices = (np.argsort(scores)+1)[::-1]
sorted_scores = (np.sort(scores))[::-1]
print('PageRank值从小到大排序：',sorted_indices)
print('PageRank值从小到大排序：',sorted_scores)

In [ ]:
ls=np.zeros(10)
ls[2:4]+=2
print(ls)

# Block-Stripe优化

In [ ]:
block_size=2000 # 每次迭代的块大小

block_num=node_num//block_size

remainder=node_num%block_size

if remainder!=0:
    block_num+=1

stripes = [ {} for _ in range(block_num)]

length=[0 for _ in range(node_num)]

# 初始化稀疏矩阵
for edge in G:
    to_node=edge[1]-1
    from_node=edge[0]-1
    index=to_node//block_size
    if from_node not in stripes[index]:
        stripes[index][from_node]=[]
    length[from_node]+=1

# 处理dead-end
for i in range(node_num):
    is_de=True
    for j in range(block_num):
        if i in stripes[j]:
            is_de=False
            break
    if is_de:
        length[i]=node_num
        for j in range(block_num):
            stripes[j][i]=[j*block_size+m for m in range(block_size)]

scores=np.ones((node_num))/node_num

new_scores=np.zeros((node_num))
e=1
beta=0.8
while e>1e-5:
    e=0;
    for i in range(block_num):
        new_scores[i*block_size:(i+1)*block_size]=(1-beta)/node_num
        for from_node in stripes[i]:
            for to_node in stripes[i][from_node]:
                new_scores[to_node]+=beta*scores[from_node]/length[from_node]
        e+=sum(abs(new_scores[i*block_size:(i+1)*block_size]-scores[i*block_size:(i+1)*block_size]))
    
    # 处理剩余部分
    if remainder!=0:
        new_scores[block_num*block_size:]=(1-beta)/node_num
        for from_node in stripes[block_num-1]:
            for to_node in stripes[block_num-1][from_node]:
                new_scores[to_node]+=beta*scores[from_node]/length[from_node]
        e+=sum(abs(new_scores[block_num*block_size:]-scores[block_num*block_size:]))
    scores=np.copy(new_scores)

print(scores)

In [ ]:
print(node_num)